In [1]:
import pandas as pd
from pathlib import Path
import yaml
import numpy as np

from table_utils import get_frames_of_choices_raw
from table_utils import get_sampled_choices

## The pagewise dataset serves as the foundation for user preferences

# 1. Sample pages 
- stratified by `train`, `test`, `val`
- pages 0,1,2 only (to ensure alignment as pages are assigned in a data-driven fashion - potentially errorneous - for marker/nougat/grobid)

# 2. Sample & sort binary pairings 

## 2.1 Comparison variants
Two groups seeing `300` binary comparisons per user (in two variants).
- first pairing most effort (new image, 2 new texts)
- second pairing (same image, one option similar) -> less effort, easier to compare
- Group $A$: `5` pairings per image (--> $5 \cdot 60$) (identified by even user id)
- Group $B$: `3` pairings per image (--> $3 \cdot 100$) (identified by odd user id)
- assume 12 users (but calculate with worst-case 6 completing it)
   --> `n_train=1000`, `n_val=n_test=600`
- stratified by `mean_bleu` as many pages have very low score (indicates wrong split of pages I'd say)
- --> 50% high mean_bleu (>0.6) and 50% high mean_bleu (0.6> bleu >0.4)
  
### 2.2 `train`, `val` and `test`
- split: 60%, 20%, 20% (since `val` is over-represented, that data could be merged into `train` yielding 80%/20%, respectively)
- --> sample 1500 for train, 1000 val, 1000 test (+ quality control), 
- add `GPT-4o` occasionally to get idea of how much people like SOTA (but closed-source parsing)

### 2.3 Sample 
- randomly assign `group` letter to each row (indicating how many to sample)

## 3. Identification of File
Image ID: identifies each image uniquely (to assign to PDF, it's page and what subset the PDF initially belonged to when pulled).
Subset id may appear reduntant but split could change in future.
```
IMAGE_ID: {pdf_path_id:6}{page_id:2}{subset_id:1}
```
Basically a mapping from (`path`,`page`) $\mapsto$ `IMAGE_ID`.
Once an image is assigned to a user, text etc. additional metadata is needed. Choice id does that.
It identifies each binary pairing of texts (& position) as well as the displayed image uniquely in 6+2+1+2+4+1+1=17 digits).
The image ID can (and will occure) multiple times (same page image but different parsers) or the same choice is shown to different users etc.
```
CHOICE_ID: {pdf_path_id:6}{page_id:2}{subset_id:1}{user_group_id:2}{occurence_id:4}{left_text_id:1}{right_text_id:1}
```

In [2]:
%%time

# get raw
df_train, df_test, df_val = get_frames_of_choices_raw()

# - const
#df_train_out = get_sampled_choices(df_train, 'train')
#df_test_out = get_sampled_choices(df_train, 'test')
#df_val_out = get_sampled_choices(df_train, 'val')

/home/siebenschuh/Projects/dataprep/code/user_preference_survey/table_utils.py:149: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p_df, sep='|')


CPU times: user 58.6 s, sys: 6.39 s, total: 1min 4s
Wall time: 1min 5s


In [3]:
df_train_out = get_sampled_choices(df_train, 'train')
df_val_out = get_sampled_choices(df_train, 'val')

len(df)=1500, len(train_5_tuples): 15000, len(train_3_tuples)=15000
len(df)=1500, len(train_5_tuples): 15000, len(train_3_tuples)=15000


In [ ]:
len(set(df_train['path']))

```
IMAGE_ID: {pdf_path_id:6}{page_id:2}{subset_id:1}
CHOICE_ID: {pdf_path_id:6}{page_id:2}{subset_id:1}{user_group_id:2}{occurence_id:4}{left_text_id:1}{right_text_id:1}
```

In [ ]:
df_train_out = get_sampled_choices(df_train, 'train')

In [ ]:
df_train_out

In [ ]:
image

In [ ]:
print(DF.iloc[100]['left'])